### This Bot exports the data from SAP and adds it into a dedicated Share Point Folder

In [4]:
import win32com.client as wincli
import subprocess
import time

In [5]:
# open SAP Logon
subprocess.Popen(r"C:\\Program Files (x86)\SAP\\FrontEnd\SAPgui\saplogon.exe")
time.sleep(5)
SapGuiAuto = wincli.GetObject('SAPGUI')
application = SapGuiAuto.GetScriptingEngine
connection = application.OpenConnection("USDEV",True)

# login
session = connection.Children(0)
session.findbyId("wnd[0]").maximize
session.findById("wnd[0]/usr/txtRSYST-MANDT").text = "800" # Client
session.findById("wnd[0]/usr/txtRSYST-BNAME").text = "JESAI"    # User
session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = "jaisai2025"  # Password
session.findById("wnd[0]/usr/txtRSYST-LANGU").text = "EN"   # Language
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [6]:
# navigate to transaction code
session.findById("wnd[0]/tbar[0]/okcd").text = "/nSQVI" # Transaction code
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [ ]:
# Entering the View to Execute
session.findById("wnd[0]/usr/ctxtRS38R-QNUM").text = "STOCK_QUERY"
# session.findById("wnd[0]/usr/btnP1").press
session.findById("wnd[0]").sendVKey(8) # F8 - Execute

session.findById("wnd[0]").sendVKey(8) # F8



In [8]:
# Try possible grid control IDs
possible_ids = [
	"wnd[0]/usr/cntlGRID1/shellcont/shell",
	"wnd[0]/usr/cntlALV_CONTAINER_1/shellcont/shell",
	"wnd[0]/usr/cntlCONTAINER/shellcont/shell"
]

grid = None
for grid_id in possible_ids:
	try:
		grid = session.findById(grid_id)
		break
	except Exception:
		continue

if grid is None:
	raise Exception("ALV grid control not found. Tried: " + ", ".join(possible_ids))

grid.pressToolbarContextButton("&MB_EXPORT")  # open 'List -> Export' menu on the ALV toolbar
grid.selectContextMenuItem("&XXL")     

In [9]:
# Adding Time Stamp as file name
from datetime import datetime
# Function to create time Stamp and add as Text
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
FileName = "Stock_Export"
concat = FileName+"_"+timestamp

session.findById("wnd[1]/usr/ssubSUB_CONFIGURATION:SAPLSALV_GUI_CUL_EXPORT_AS:0512/txtGS_EXPORT-FILE_NAME").text = concat

# Clicking Export Button
session.findById("wnd[0]").sendVKey(0) # Enter

In [10]:
# Saving File in Destination Folder
session.findById("wnd[1]/usr/ctxtDY_PATH").text = r"C:\\Users\\JeSai\\OneDrive - Boviet Solar USA\\Boviet IT Team - Boviet IT Team Channel\\Intern\\Je\\Stock Reports"

session.findById("wnd[0]").sendVKey(0) # Enter